In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
pd.set_option('display.max_colwidth', 300)


## Read the data

In [ ]:
df=pd.read_csv('/files/data/cleansed.csv', sep=',')
df['created'] = pd.to_datetime(df['created'], unit='s')
df



## What does moderated mean?

In [ ]:
df.reset_index().groupby(["moderated"]).count()['cid']

In [ ]:
df.reset_index().groupby(["moderated"]).count()['cid'].plot.barh()

In [ ]:
df.groupby(["moderated"]).count()['cid'].sum()

In [ ]:
count_moderated = df[df['moderated'] > 1].groupby(["moderated"]).count()['cid'].sum()
print('Moderated comments:', count_moderated)



## Search for users with many comments

In [ ]:
df.groupby('uid').count().cid.hist(bins=100)

In [ ]:
df.groupby('uid').filter(lambda x: len(x) > 10)



## Inspect a user

In [ ]:
df.loc[df['uid'] == '444042']



## Find users with moderated comments

In [ ]:
len(df[df['moderated'] > 1])

In [ ]:
df[df['moderated'] > 1]



## Find original comments

In [ ]:
# Get all moderated cids
cids = df[df.moderated > 1].cid.as_matrix()

# Filter for cids that appear at least twice
cids_at_least_twice = df[df['cid'].isin(cids)].groupby('cid').filter(lambda x: len(x) > 1).cid.as_matrix()

df[df['cid'].isin(cids_at_least_twice)].sort_values(by=['cid', 'created'])[['cid', 'comment', 'created']]

In [ ]:
df[df.cid == '5808477']



## When where moderated comments posted?

In [ ]:
tohist_df = df[df['cid'].isin(cids_at_least_twice)].drop_duplicates(subset='cid')
tohist_df["date"] = tohist_df["created"].astype("datetime64[ns]")

pd.options.mode.chained_assignment = None
tohist_df.groupby(tohist_df["date"].dt.day)['cid'].count().plot(kind="bar")

In [ ]:
tohist_df = df[df['cid'].isin(cids_at_least_twice)].drop_duplicates(subset='cid')
tohist_df["date"] = tohist_df["created"].astype("datetime64[ns]")

pd.options.mode.chained_assignment = None
tohist_df.groupby(tohist_df["date"].dt.hour)['cid'].count().apply(lambda x: x/120).plot(kind="bar")



## When were comments posted in general?

In [ ]:
tohist_df = df.drop_duplicates(subset='cid')
tohist_df["date"] = tohist_df["created"].astype("datetime64[ns]")

pd.options.mode.chained_assignment = None
tohist_df.groupby(tohist_df["date"].dt.hour)['cid'].count().apply(lambda x: x/700).plot(kind="bar")

In [ ]:
tohist_df = df[df['cid'].isin(cids_at_least_twice)].drop_duplicates(subset='cid')
tohist_df["date"] = tohist_df["created"].astype("datetime64[ns]")

moderated_df = tohist_df.groupby(tohist_df["date"].dt.hour)['cid'].count().apply(lambda x: x/120)

tohist_df = df.drop_duplicates(subset='cid')
tohist_df["date"] = tohist_df["created"].astype("datetime64[ns]")

unmoderated_df = tohist_df.groupby(tohist_df["date"].dt.hour)['cid'].count().apply(lambda x: x/700)

(moderated_df-unmoderated_df).plot()



## Search for duplicates

In [ ]:
df[df.duplicated('comment', False)].sort_values(by=['comment'])[['cid', 'comment']]

In [ ]:
duplicates = df[df.duplicated('comment')].sort_values(by=['comment'])[['cid', 'comment']]
duplicates[duplicates["comment"].str.contains('<em>')==False]

In [ ]:
duplicates = df[df.duplicated('comment')].sort_values(by=['comment'])[['cid', 'comment']]
len(duplicates[duplicates["comment"].str.contains('<em>')==False])



## Compare posts and replies

In [ ]:
print("Comments with parent:", len(df[df.pid > 0]))
print("Comments without parent:", len(df[df.pid == 0]))

print('-'*30)
df_no_moderates = duplicates[duplicates["comment"].str.contains('<em>')==True]

duplicates = df[df.duplicated('comment')].sort_values(by=['comment'])
print("Duplicates with parent:", len(duplicates[duplicates.pid > 0]))
print("Duplicates without parent:", len(duplicates[duplicates.pid == 0]))

print('-'*30)

moderated = df[df['moderated'] > 1]

print("Moderated with parent:", len(moderated[moderated.pid > 0]))
print("Moderated without parent:", len(moderated[moderated.pid == 0]))



# Hate Speech

In [ ]:
moderated_cids = df[df.moderated > 1].cid.as_matrix()
moderated_all = df[df['cid'].isin(moderated_cids)]
moderated = moderated_all[moderated_all['moderated'] == 1]

duplicate_cids = df[df.duplicated('comment')].sort_values(by=['comment'])['cid'].as_matrix()

# Filter for cids that appear at least twice
hate_comments = moderated[~moderated['cid'].isin(duplicate_cids)]
hate_comments



## Most frequent words in hate speech

In [ ]:
## in General
pd.Series(' '.join(hate_comments['comment']).lower().split()).value_counts()[:50]

In [ ]:
!pip install nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
from collections import Counter
import pandas as pd
import nltk

top_N = 50

hate_df = hate_comments[['comment']]
hate_df['comment'] = hate_df['comment'].str.replace('[-./,;!-!:()?]','')


stopwords = nltk.corpus.stopwords.words('german')
# RegEx for stopwords
RE_stopwords = r'\b(?:{})\b'.format('|'.join(stopwords))
# replace '|'-->' ' and drop all stopwords
words = (hate_df.comment
           .str.lower()
           .replace([r'\|', RE_stopwords], [' ', ''], regex=True)
           .str.cat(sep=' ')
           .split()
)

# generate DF out of Counter
rslt = pd.DataFrame(Counter(words).most_common(top_N),
                    columns=['Word', 'Frequency']).set_index('Word')
print(rslt)

# plot
rslt.head(15).plot.bar(rot=0, figsize=(16,10), width=0.8)

In [ ]:
from collections import Counter
import pandas as pd
import nltk

top_N = 50

all_df = df[['comment']]
all_df['comment'] = all_df['comment'].str.replace('[-./,;!-!:()?]','')


stopwords = nltk.corpus.stopwords.words('german')
# RegEx for stopwords
RE_stopwords = r'\b(?:{})\b'.format('|'.join(stopwords))
# replace '|'-->' ' and drop all stopwords
words = (all_df.comment
           .str.lower()
           .replace([r'\|', RE_stopwords], [' ', ''], regex=True)
           .str.cat(sep=' ')
           .split()
)

# generate DF out of Counter
all_rslt = pd.DataFrame(Counter(words).most_common(top_N),
                    columns=['Word', 'Frequency']).set_index('Word')
print(rslt)

# plot
all_rslt.head(15).plot.bar(rot=0, figsize=(16,10), width=0.8)

In [ ]:
pd.concat([all_rslt, rslt], axis=1, join='inner')

In [ ]:
pd.concat([all_rslt.reset_index(), rslt.reset_index()], axis=1, join='inner')

In [ ]:
df.drop_duplicates(subset='subject')
